In [16]:
import os
import glob
import pandas as pd

RUTA_LOGS = "."
RUTA_TEST = "."   # donde están los *_test.csv

archivos_xlsx = [
    f for f in glob.glob(os.path.join(RUTA_LOGS, "*.xlsx"))
    if not os.path.basename(f).startswith("~$")
]

archivos_test = glob.glob(os.path.join(RUTA_TEST, "*_test.csv"))

# ---- cargar distribuciones de test ----
dist_test = {}
for ruta in archivos_test:
    nombre = os.path.basename(ruta)
    dataset = nombre.split("_tdataset")[0]
    df_test = pd.read_csv(ruta)
    col_clase = df_test.columns[-1]
    dist_test[dataset] = df_test[col_clase].value_counts()

# ---- análisis principal ----
for ruta in archivos_xlsx:
    nombre_archivo = os.path.basename(ruta)
    dataset = (
        nombre_archivo
        .replace("log_pcsmote_x_muestra_", "")
        .replace(".xlsx", "")
    )

    df = pd.read_excel(ruta, engine="openpyxl")

    columnas = {
        "configuracion",
        "clase_objetivo",
        "synthetics_from_this_seed",
        "es_semilla_valida"
    }
    if not columnas <= set(df.columns):
        raise ValueError(f"Columnas faltantes en {nombre_archivo}")

    print("\n" + "=" * 100)
    print(f"Dataset: {dataset}")
    print("=" * 100)

    for configuracion, bloque in df.groupby("configuracion"):

        print(f"\n{configuracion}")
        print("-" * len(configuracion))

        resumen = (
            bloque
            .groupby("clase_objetivo")
            .agg(
                SG=("synthetics_from_this_seed", "sum"),
                SV=("es_semilla_valida", "sum")
            )
            .sort_index()
        )

        # imprimir por clase
        for clase, fila in resumen.iterrows():
            sg = int(fila["SG"])
            sv = int(fila["SV"])
            ratio = sg / sv if sv > 0 else 0
            print(f"{clase:<6} SV={sv:<3} SG={sg:<4} SG/SV={ratio:.2f}")

        # métricas globales
        total_sg = resumen["SG"].sum()
        cobertura = (resumen["SG"] > 0).sum()
        total_clases = len(resumen)

        print(f"\n  ▶ Total sintéticos (SG): {int(total_sg)}")
        print(f"  ▶ Cobertura de clases  : {cobertura}/{total_clases}")

        if total_sg == 0:
            print("  ⚠ Configuración inválida (SG = 0)")
            continue

        # ---- desalineación con test ----
        if dataset in dist_test:
            print("  ▶ Desalineación SG vs TEST:")
            for clase in resumen.index:
                sg = resumen.loc[clase, "SG"]
                ft = dist_test[dataset].get(clase, 0)
                delta = sg - ft
                print(f"     {clase:<6} SG={sg:<4} TEST={ft:<3} Δ={delta}")



Dataset: ecoli

PRD70_PR30_CPprop_UD060_UR040_Upp6000_I0
----------------------------------------
im     SV=0   SG=0    SG/SV=0.00
imU    SV=0   SG=0    SG/SV=0.00
om     SV=0   SG=0    SG/SV=0.00
pp     SV=0   SG=0    SG/SV=0.00

  ▶ Total sintéticos (SG): 0
  ▶ Cobertura de clases  : 0/4
  ⚠ Configuración inválida (SG = 0)

PRD70_PR30_CPprop_UD060_UR040_Upp6000_I1
----------------------------------------
im     SV=0   SG=0    SG/SV=0.00
imU    SV=0   SG=0    SG/SV=0.00
om     SV=0   SG=0    SG/SV=0.00
pp     SV=0   SG=0    SG/SV=0.00

  ▶ Total sintéticos (SG): 0
  ▶ Cobertura de clases  : 0/4
  ⚠ Configuración inválida (SG = 0)

PRD70_PR30_CPprop_UD060_UR040_Upp6000_I3
----------------------------------------
im     SV=0   SG=0    SG/SV=0.00
imU    SV=0   SG=0    SG/SV=0.00
om     SV=0   SG=0    SG/SV=0.00
pp     SV=0   SG=0    SG/SV=0.00

  ▶ Total sintéticos (SG): 0
  ▶ Cobertura de clases  : 0/4
  ⚠ Configuración inválida (SG = 0)

PRD70_PR40_CPent_UD060_UR045_PE60_I0
----------